In [ ]:
import numpy as np
import math
import random
import time
from scipy.optimize import minimize

#Get Angle from Location

In [ ]:
def get_robot_position_error(true_robot_position, add_error = False):
  x_error = (random.random() % 500) - 250
  y_error = (random.random() % 500) - 250
  z_error = (random.random() % 500) - 250
  #adding random error

  distance = np.sqrt((x_error ** 2) + (y_error ** 2) + (z_error ** 2))

  believed_position = (true_robot_position[0] + x_error * 2 / distance, true_robot_position[1] + y_error * 2 / distance, true_robot_position[2] + z_error * 2 / distance)

  return believed_position

def get_angle_error(true_angle, add_error = False):
  (theta, phi) = true_angle
  if add_error == True:
    angle = random.random() * 10000
    theta += 0.0872665 * np.sin(angle)
    phi += 0.0872665 * np.cos(angle)
  return (theta, phi)

def get_angle(true_robot_position, source_position, add_error = False):
  robot_position = true_robot_position
  if add_error == True:
    robot_position = get_robot_position_error(robot_position)

  delta_x = source_position[0] - robot_position[0]
  delta_y = source_position[1] - robot_position[1]
  delta_z = source_position[2] - robot_position[2]
  distance = np.sqrt((delta_x**2) + (delta_y**2) + (delta_z**2))

  if distance < 300:
    ratio = np.abs(delta_y / delta_x)
    if (delta_x > 0 and delta_y > 0):
      theta = np.arctan(ratio)
    elif (delta_x < 0 and delta_y > 0):
      theta = np.pi - np.arctan(ratio)
    elif (delta_x > 0 and delta_y < 0):
      theta = -1 * np.arctan(ratio)
    elif (delta_x < 0 and delta_y < 0):
      theta = np.arctan(ratio) + np.pi
    phi = np.arccos(delta_z / np.sqrt((delta_x ** 2) + (delta_y ** 2) + (delta_z ** 2)))

    (theta, phi) = get_angle_error((theta, phi), add_error)

    dx = np.sin(phi) * np.cos(theta)
    dy = np.sin(phi) * np.sin(theta)
    dz = np.cos(phi)

    return (dx, dy, dz)

  return (100, 100, 100)

In [ ]:
def single_line_distance(point, line):
  base_to_point = (line[0]-point[0], line[1]-point[1], line[2]-point[2]) #v-p
  unit_vector_direction = (line[3], line[4], line[5]) #This MUST be a unit vector or the entire system fails.
  cross_product = np.cross(base_to_point, unit_vector_direction)
  distance = math.sqrt(sum(pow(element, 2) for element in cross_product))
  return distance

def total_distance(point, lines):
  total_distance = 0
  for line in lines:
    total_distance += single_line_distance(point, line)
  return total_distance

def find_closest_point_to_lines(lines, room_size):
  room_w = room_size[0]
  room_l = room_size[1]
  room_h = room_size[2]

  guesses = ((0,0,room_h/2), (0,room_l,room_h/2), (room_w,0,room_h/2), (room_w,room_l,room_h/2), (room_w/2,room_l/2,room_h/2), (room_w,room_l,room_h/2), (room_w,room_l/2,room_h/2), (room_w/2,room_l,room_h/2), (room_w/2,room_l/2,room_h/2))
  results = []
  distances = []
  smallest_distance = 10 ** 100
  result = (-100, -100, -100)
  for i in range(0,len(guesses)):
    results.append(minimize(total_distance, guesses[i], args=lines))
    distances.append(total_distance(results[i].x, lines))
    if distances[i] < smallest_distance:
      smallest_distance = distances[i]
      result = results[i]

  closest_point = result.x
  return closest_point

In [ ]:
def run_test(room_size):
  start_time = time.time()

  room_w = room_size[0]
  room_l = room_size[1]
  room_h = room_size[2]

  number_of_recording_positions = int(np.floor((room_w - 50)/100) * np.floor((room_l - 50)/100))

  source = (random.random()*room_w, random.random()*room_l, random.random()*room_h)
  positions = []
  angles = []
  for i in range(50, room_w, 100):
    for j in range(50, room_l, 100):
      positions.append((i, j, 0))
      angles.append(get_angle((i, j, 0), source, add_error = True))

  source_lines = []
  for i in range(0, number_of_recording_positions, 1):
    if angles[i][0] != 100:
      source_lines.append((positions[i][0], positions[i][1], positions[i][2], angles[i][0], angles[i][1], angles[i][2]))
  estimated_source = find_closest_point_to_lines(source_lines, room_size)

  end_time = time.time()
  distance = np.sqrt(sum(d**2 for d in (source - estimated_source)))

  if distance <= 15:
    print("SUCCESS!", end_time - start_time)
    return 0, end_time - start_time

  print("Failure", source, estimated_source, distance)
  return 1, end_time - start_time

In [ ]:
number_runs = 1000

number_failed = 0
total_time = 0
for i in range(0, number_runs):
  new_fail, new_time = run_test((2000, 2000, 100))
  number_failed += new_fail
  total_time += new_time
  if (i+1)%100 == 0:
    print(i, 100 - 100*(number_failed / (i+1)))
print(100 - (number_failed * 100 / number_runs))
print(total_time / number_runs)

SUCCESS! 1.2256128787994385
SUCCESS! 1.1359479427337646
SUCCESS! 1.0100083351135254
SUCCESS! 0.9090063571929932
SUCCESS! 0.8549778461456299
SUCCESS! 4.336644887924194
SUCCESS! 0.9410126209259033
SUCCESS! 0.8756802082061768
SUCCESS! 0.7063705921173096
SUCCESS! 0.749605655670166
SUCCESS! 1.4036917686462402
SUCCESS! 4.642337322235107
SUCCESS! 1.2533924579620361
SUCCESS! 0.5405373573303223
SUCCESS! 1.0277376174926758
SUCCESS! 0.8368682861328125
SUCCESS! 1.9229967594146729


KeyboardInterrupt: 